In [1]:
import pandas as pd

In [4]:
data=pd.read_csv(r"C:\Users\sunny\custmor_support_system\data\amazon_product_review.csv")

In [5]:
data.head()

,product_id,product_title,rating,summary,review
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,1-more flexible2-bass is very high3-sound clar...
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Super sound and good looking I like that prize
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,Very much satisfied with the device at this pr...
3,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,"Nice headphone, bass was very good and sound i..."
4,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Sound quality super battery backup super quali...


In [6]:
data.columns

Index(['product_id', 'product_title', 'rating', 'summary', 'review'], dtype='object')

In [8]:
data["review"][0]

"1-more flexible2-bass is very high3-sound clarity is good 4-battery back up to 6 to 8 hour's 5-main thing is fastest charging system is available in that. Only 20 min charge and get long up to 4 hours back up 6-killing look awesome 7-for gaming that product does not support 100% if you want for gaming then I'll recommend you please don't buy but you want for only music then this product is very well for you.. 8-no more wireless headphones are comparing with that headphones at this pric..."

In [10]:
data["rating"][:5]

0    5
1    5
2    5
3    5
4    5
Name: rating, dtype: int64

In [11]:
data["product_title"][:5]

0    BoAt Rockerz 235v2 with ASAP charging Version ...
1    BoAt Rockerz 235v2 with ASAP charging Version ...
2    BoAt Rockerz 235v2 with ASAP charging Version ...
3    BoAt Rockerz 235v2 with ASAP charging Version ...
4    BoAt Rockerz 235v2 with ASAP charging Version ...
Name: product_title, dtype: object

In [12]:
data["summary"][:5]

0    Terrific purchase
1    Terrific purchase
2               Super!
3               Super!
4    Terrific purchase
Name: summary, dtype: object

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

In [3]:
groq_model=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [4]:

groq_model.invoke("what is capital of USA")

AIMessage(content='<think>\n\n</think>\n\nThe capital of the United States is **Washington, D.C.** (short for Washington District of Columbia).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 8, 'total_tokens': 35, 'completion_time': 0.106900445, 'prompt_time': 0.000142947, 'queue_time': 0.054835693000000005, 'total_time': 0.107043392}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-ad6e819e-d62b-4094-ab49-527affc6a435-0', usage_metadata={'input_tokens': 8, 'output_tokens': 27, 'total_tokens': 35})

In [2]:
import os
if "GROQ_API_KEY" in os.environ:
    print("yes")

yes


In [4]:
import requests
from bs4 import BeautifulSoup
import re

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re

In [24]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time
import csv
import re

def flipkart_scrape_to_csv(query, output_file="flipkart_products.csv"):
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")

    driver = uc.Chrome(options=options)

    search_url = f"https://www.flipkart.com/search?q={query.replace(' ', '+')}"
    driver.get(search_url)

    time.sleep(6)

    # Scroll to load more
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    html = driver.page_source
    print(html[:2000])  # DEBUG: check HTML response

    soup = BeautifulSoup(html, 'html.parser')
    driver.quit()

    products = []

    for item in soup.select("div._1AtVbE"):
        link_tag = item.select_one("a[href*='/p/']")
        title_tag = item.select_one("div._4rR01T") or item.select_one("a.s1Q9rs")
        rating_tag = item.select_one("div._3LWZlK")
        review_tag = item.select_one("span._2_R_DZ")

        if link_tag and title_tag:
            link = link_tag["href"]
            product_id = re.findall(r"/p/itm[0-9a-zA-Z]+", link)
            product_id = product_id[0].split("/")[-1] if product_id else "N/A"

            title = title_tag.get_text(strip=True)
            rating = rating_tag.get_text(strip=True) if rating_tag else "No rating"
            review = "No reviews"

            if review_tag:
                match = re.search(r"(\d+,?\d*)\s+Ratings\s*&\s*(\d+,?\d*)\s+Reviews", review_tag.get_text())
                if match:
                    review = match.group(2) + " reviews"

            products.append([product_id, title, rating, review])

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["product_id", "product_title", "rating", "review"])
        writer.writerows(products)

    print(f"✅ Exported {len(products)} products to {output_file}")

flipkart_scrape_to_csv("smartphone under 20000")


<html lang="en"><head><script type="text/javascript" async="async" src="https://flipkart.d1.sc.omtrdc.net/id?d_visid_ver=1.5.4&amp;callback=s_c_il%5B0%5D._setAnalyticsFields&amp;mcorgid=17EB401053DAF4840A490D4C%40AdobeOrg&amp;mid=72830571568854856153932040467330559931"></script><style id="react-native-stylesheet"></style><script type="text/javascript" async="async" src="https://dpm.demdex.net/id?d_visid_ver=1.5.4&amp;d_rtbd=json&amp;d_ver=2&amp;d_orgid=17EB401053DAF4840A490D4C%40AdobeOrg&amp;d_nsid=0&amp;d_cb=s_c_il%5B0%5D._setMarketingCloudFields"></script><link href="https://rukminim2.flixcart.com" rel="preconnect"><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/atlas.chunk.8dd48d.css"><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.c48a12.css"><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.e4e719.css"> <meta http-eq

In [17]:
products

[]

In [18]:
# Test
products = flipkart_selenium_scrape("iphone 16")

In [19]:
products

[]

In [13]:
products

[]

In [ ]:

for p in products[:5]:
    print(p)


In [3]:
import asyncio
import csv
import re
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def scrape_flipkart_async(query, output_file="flipkart_playwright_products.csv"):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        search_url = f"https://www.flipkart.com/search?q={query.replace(' ', '+')}"
        await page.goto(search_url)
        await page.wait_for_selector("div._1AtVbE", timeout=10000)

        for _ in range(3):
            await page.mouse.wheel(0, 10000)
            await asyncio.sleep(2)

        content = await page.content()
        await browser.close()

    soup = BeautifulSoup(content, "html.parser")
    products = []

    for item in soup.select("div._1AtVbE"):
        title_tag = item.select_one("div._4rR01T") or item.select_one("a.s1Q9rs")
        rating_tag = item.select_one("div._3LWZlK")
        review_tag = item.select_one("span._2_R_DZ")
        link_tag = item.select_one("a[href*='/p/']")

        if title_tag and link_tag:
            title = title_tag.get_text(strip=True)
            rating = rating_tag.get_text(strip=True) if rating_tag else "No rating"
            review = "No reviews"

            if review_tag:
                match = re.search(r"\d+ Reviews", review_tag.get_text())
                if match:
                    review = match.group(0)

            product_id = re.findall(r"/p/itm[0-9a-zA-Z]+", link_tag["href"])
            product_id = product_id[0].split("/")[-1] if product_id else "N/A"

            products.append([product_id, title, rating, review])
    print(products)

    # Save to CSV
    with open(output_file, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["product_id", "product_title", "rating", "review"])
        writer.writerows(products)

    print(f"✅ Exported {len(products)} products to {output_file}")



In [4]:
await scrape_flipkart_async("smartphone under 15000")


NotImplementedError: 

In [2]:
# Export to CSV
with open("flipkart_from_google.csv", "w", newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["title", "link"])
    writer.writeheader()
    writer.writerows(products)

print(f"✅ Exported {len(products)} Flipkart products via SerpAPI")

search_flipkart_products("laptop under 50000")

NameError: name 'products' is not defined